In [ ]:
from google.colab import drive

In [ ]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dirname = os.path.join(os.getcwd(), 'drive','MyDrive','Archivos para ML','FrutasExamenSmall')
imgpath = dirname + os.sep

images = []
directories = []
dircount = []
prevRoot=''
cant=0
print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0

dircount.append(cant)
print('Elementos del arreglo de imagenes: ',len(images))
dircount = dircount[1:]
dircount[2]=dircount[2]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /content/drive/MyDrive/Archivos para ML/FrutasExamenSmall/
/content/drive/MyDrive/Archivos para ML/FrutasExamenSmall/Pera 1
/content/drive/MyDrive/Archivos para ML/FrutasExamenSmall/Palta 30
/content/drive/MyDrive/Archivos para ML/FrutasExamenSmall/Mango 30
Elementos del arreglo de imagenes:  90
Directorios leidos: 3
Imagenes en cada directorio [30, 30, 30]
suma Total de imagenes en subdirs: 90


In [ ]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

frutas=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    frutas.append(name[len(name)-1])
    indice=indice+1

y = np.array(labels)
X = np.array(images, dtype=np.uint8)

classes = np.unique(y)
nClasses = len(classes)
print('Total de salidas : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  90
0 Pera
1 Palta
2 Mango
Total de salidas :  3
Output classes :  [0 1 2]


In [ ]:
#Crear grupos de entranmiento y testeo
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.33)
print('(Datos de entranamieno, ancho, alto, rgb) : ', train_X.shape, train_Y.shape)
print('(Datos de testeo, ancho, alto, rgb) : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Aplicando one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)


(Datos de entranamieno, ancho, alto, rgb) :  (60, 75, 100, 3) (60,)
(Datos de testeo, ancho, alto, rgb) :  (30, 75, 100, 3) (30,)


In [ ]:
INIT_LR = 1e-3
epochs = 40
batch_size = 32

sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(75,100,3)))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))

sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5))
sport_model.add(Dense(nClasses, activation='softmax'))

sport_model.summary()

sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 75, 100, 32)       896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 75, 100, 32)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 38, 50, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 38, 50, 32)        0         
                                                                 
 flatten (Flatten)           (None, 60800)             0         
                                                                 
 dense (Dense)               (None, 32)                1945632   
                                                        

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adagrad.py:84: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
sport_train_dropout = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))



Epoch 1/40
2/2 [==============================] - 9s 381ms/step - loss: 1.1430 - accuracy: 0.3542 - val_loss: 1.2167 - val_accuracy: 0.4167
Epoch 2/40
2/2 [==============================] - 0s 35ms/step - loss: 1.1554 - accuracy: 0.5208 - val_loss: 1.0175 - val_accuracy: 0.4167
Epoch 3/40
2/2 [==============================] - 0s 34ms/step - loss: 1.1790 - accuracy: 0.4167 - val_loss: 1.2089 - val_accuracy: 0.4167
Epoch 4/40
2/2 [==============================] - 0s 35ms/step - loss: 1.2311 - accuracy: 0.3958 - val_loss: 1.2071 - val_accuracy: 0.4167
Epoch 5/40
2/2 [==============================] - 0s 42ms/step - loss: 1.1256 - accuracy: 0.3958 - val_loss: 1.0350 - val_accuracy: 0.4167
Epoch 6/40
2/2 [==============================] - 0s 33ms/step - loss: 1.0752 - accuracy: 0.3958 - val_loss: 1.0151 - val_accuracy: 0.6667
Epoch 7/40
2/2 [==============================] - 0s 34ms/step - loss: 1.0638 - accuracy: 0.4583 - val_loss: 1.0148 - val_accuracy: 0.7500
Epoch 8/40
2/2 [==========

In [ ]:
test_eval = sport_model.evaluate(test_X, test_Y_one_hot, verbose=1)

print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

1/1 [==============================] - 0s 84ms/step - loss: 0.7977 - accuracy: 0.8333
Test loss: 0.7977226376533508
Test accuracy: 0.8333333134651184
